# CycleGAN: Human Faces to Anime Style
Model 1 of **Genshinfy-UwU**. This model is used for style-transfer based pairing in creating a dataset for training a siamese network (Model 2 of **Genshinfy-UwU**) to compare similarity between human and anime faces.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import matplotlib.pyplot as plt # plotting


import tensorflow as tf
import keras
from keras import layers, ops


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

os.environ["KERAS_BACKEND"] = "tensorflow"
autotune = tf.data.AUTOTUNE

## Creating a Custom Dataset

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 1

In [ ]:
def load_image(path):
    image = tf.io.read_file(path)
    #if image.shape == ():
    #    print(f"Warning: Could not load image at {path}")
    #    return tf.zeros([IMG_HEIGHT, IMG_WIDTH, 3]) #return empty tensor
    image = tf.image.decode_jpeg(image, channels=3) #remove RGBA from GenshinImpact
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = (image / 127.5) - 1
    return image

In [ ]:
def load_dataset(path):
    image_paths = tf.data.Dataset.list_files(
        os.path.join(path, '*'),
        shuffle=True
    )
    dataset = image_paths.map(
        load_image,
        num_parallel_calls=autotune
    )
    dataset = dataset.batch(BATCH_SIZE).prefetch(autotune)
    return dataset

In [ ]:
AnimeFaces = load_dataset('/kaggle/input/animefacedataset/images')

In [ ]:
CelebA = load_dataset('/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba')

In [ ]:
CycleDataset = tf.data.Dataset.zip((
    CelebA.shuffle(buffer_size=256).take(1024),
    AnimeFaces.shuffle(buffer_size=256).take(1024)
))

We can visualize some of the images in our dataset.

In [ ]:
_, ax = plt.subplots(2, 2, figsize=(12, 12))
for i, samples in enumerate(CycleDataset.take(2)):
    real = (((samples[0][0] * 127.5) + 127.5).numpy()).astype(np.uint8)
    fake = (((samples[1][0] * 127.5) + 127.5).numpy()).astype(np.uint8)
    ax[i, 0].imshow(real)
    ax[i, 1].imshow(fake)
plt.show()

## Define Custom Layers

In [ ]:
class ReflectionPadding2D(layers.Layer):
    def __init__(self, padding=(1,1), **kwargs):
        self.padding = tuple(padding)
        super().__init__(**kwargs)

    def call(self, inputs, mask=None):
        pad_width, pad_height = self.padding
        pad_tensor = [
            [0, 0],
            [pad_height, pad_height],
            [pad_width, pad_width],
            [0, 0]
        ]
        return ops.pad(inputs, pad_tensor, mode="REFLECT")

In [ ]:
from keras.initializers import RandomNormal

KI = RandomNormal(mean=0.0, stddev=0.02)
GI = RandomNormal(mean=0.0, stddev=0.02)

In [ ]:
def ResidualBlock(
    x,
    activation,
    kernel_initializer = KI,
    kernel_size = (3, 3),
    strides = (1, 1),
    padding = "valid",
    gamma_initializer = GI,
    use_bias = False
):
    dim = x.shape[-1]
    inputs = x
    x = ReflectionPadding2D()(inputs)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides = strides,
        kernel_initializer = kernel_initializer,
        padding = padding,
        use_bias = use_bias,
    )(x)
    x = layers.GroupNormalization(
        groups = 1,
        gamma_initializer = gamma_initializer,
    )(x)
    x = layers.add([inputs, x])
    return x

In [ ]:
def DownSample(
    x,
    filters,
    activation,
    kernel_initializer = KI,
    kernel_size = (3, 3),
    strides = (2, 2),
    padding = "same",
    gamma_initializer = GI,
    use_bias = False,
):
    x = layers.Conv2D(
        filters,
        kernel_size,
        strides = strides,
        kernel_initializer = kernel_initializer,
        padding = padding,
        use_bias = use_bias,
    )(x)
    x = layers.GroupNormalization(
        groups = 1,
        gamma_initializer = gamma_initializer,
    )(x)
    if activation: x = activation(x)
    return x

In [ ]:
def UpSample(
    x,
    filters,
    activation,
    kernel_initializer = KI,
    kernel_size = (3, 3),
    strides = (2, 2),
    padding = "same",
    gamma_initializer = GI,
    use_bias = False,
):
    x = layers.Conv2DTranspose(
        filters,
        kernel_size,
        strides = strides,
        kernel_initializer = kernel_initializer,
        padding = padding,
        use_bias = use_bias,
    )(x)
    x = layers.GroupNormalization(
        groups = 1,
        gamma_initializer = gamma_initializer,
    )(x)
    if activation: x = activation(x)
    return x

## Build a ResNet Model (CycleGAN Discriminator)

In [ ]:
def ResNETGenerator(
    filters = 64,
    downsampling_blocks = 2,
    residual_blocks = 9,
    upsampling_blocks = 2,
    gamma_initializer = GI,
    name = "ResNETGenerator"
):
    image_input = layers.Input(shape=(256, 256, 3), name=name+"_img_input")
    x = ReflectionPadding2D(padding=(3, 3))(image_input)
    x = layers.Conv2D(
        filters,
        (7, 7),
        kernel_initializer = KI,
        use_bias = False,
    )(x)
    x = layers.GroupNormalization(
        groups = 1,
        gamma_initializer = gamma_initializer,
    )(x)
    x = layers.Activation("relu")(x)

    # Downsample
    for _ in range(downsampling_blocks):
        filters *= 2
        x = DownSample(
            x,
            filters,
            activation = layers.Activation("relu")
        )

    # Residual Blocks
    for _ in range(residual_blocks):
        x = ResidualBlock(
            x,
            activation = layers.Activation("relu")
        )

    # Upsample
    for _ in range(upsampling_blocks):
        filters //= 2
        x = UpSample(
            x,
            filters,
            activation = layers.Activation("relu")
        )

    x = ReflectionPadding2D(padding=(3,3))(x)
    x = layers.Conv2D(
        3,
        (7, 7),
        padding = "valid",
    )(x)
    x = layers.Activation("tanh")(x)

    return keras.models.Model(image_input, x, name=name)

In [ ]:
TestResNET = ResNETGenerator()
TestResNET.summary()

## Build the CycleGAN Discriminator

In [ ]:
def Discriminator(
    filters = 64,
    kernel_initializer = KI,
    downsampling = 3,
    name = "Discriminator",
):
    image_input = layers.Input(shape=(256,256,3), name=name+"_img_input")
    x = layers.Conv2D(
        filters,
        (4, 4),
        strides=(2, 2),
        padding="same",
        kernel_initializer=kernel_initializer,
    )(image_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for block in range(downsampling):
        num_filters *= 2
        if block < 2:
            x = DownSample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(2, 2),
            )
        else:
            x = DownSample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(1, 1),
            )

    x = layers.Conv2D(
        1,
        (4, 4),
        strides=(1, 1),
        padding="same",
        kernel_initializer=kernel_initializer,
    )(x)

    return keras.models.Model(image_input, x, name=name)

In [ ]:
TestDiscriminator = Discriminator()
TestDiscriminator.summary()

Get the Generators and Discriminators

In [ ]:
# Get the generators
gen_G = ResNETGenerator(name="generator_G")
gen_F = ResNETGenerator(name="generator_F")

# Get the discriminators
disc_X = Discriminator(name="discriminator_X")
disc_Y = Discriminator(name="discriminator_Y")

## Build the CycleGAN
**Genshinfy-UwU Pipeline**    
The CycleGAN is to generate anime versions of human faces to serve as (human, anime) data pairs to the siamese network. This is because the siamese network is used for identifying the Genshin character most similar to the user's image. The style transfer model will then use these two images as input to Genshinfy the user.

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
class GenshinfySiameseDataGenerator(keras.Model):
    def __init__(
        self,
        generator_G,
        generator_F,
        discriminator_X,
        discriminator_Y,
        lambda_cycle = 10.0,
        lambda_identity = 0.5,
    ):
        super().__init__()
        self.gen_G = generator_G
        self.gen_F = generator_F
        self.disc_X = discriminator_X
        self.disc_Y = discriminator_Y
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity

    def call(self, inputs):
        return (
            self.disc_X(inputs),
            self.disc_Y(inputs),
            self.gen_G(inputs),
            self.gen_F(inputs),
        )

    def compile(
        self,
        gen_G_optimizer,
        gen_F_optimizer,
        disc_X_optimizer,
        disc_Y_optimizer,
        gen_loss_fn,
        disc_loss_fn,
    ):
        super().compile()
        self.gen_G_optimizer = gen_G_optimizer
        self.gen_F_optimizer = gen_F_optimizer
        self.disc_X_optimizer = disc_X_optimizer
        self.disc_Y_optimizer = disc_Y_optimizer
        self.generator_loss_fn = gen_loss_fn
        self.discriminator_loss_fn = disc_loss_fn
        self.cycle_loss_fn = keras.losses.MeanAbsoluteError()
        self.identity_loss_fn = keras.losses.MeanAbsoluteError()

    def train_step(self, inputs):
        real_x, real_y = inputs # x is real, y is genshin
        with tf.GradientTape(persistent=True) as tape:
            fake_y = self.gen_G(real_x, training=True)
            fake_x = self.gen_F(real_y, training=True)
            cycled_x = self.gen_F(fake_y, training=True)
            cycled_y = self.gen_G(fake_x, training=True)

            # Identity mapping
            same_x = self.gen_F(real_x, training=True)
            same_y = self.gen_G(real_y, training=True)

            # Discriminator output
            disc_real_x = self.disc_X(real_x, training=True)
            disc_fake_x = self.disc_X(fake_x, training=True)

            disc_real_y = self.disc_Y(real_y, training=True)
            disc_fake_y = self.disc_Y(fake_y, training=True)

            # Generator adversarial loss
            gen_G_loss = self.generator_loss_fn(disc_fake_y)
            gen_F_loss = self.generator_loss_fn(disc_fake_x)

            # Generator cycle loss
            cycle_loss_G = self.cycle_loss_fn(real_y, cycled_y) * self.lambda_cycle
            cycle_loss_F = self.cycle_loss_fn(real_x, cycled_x) * self.lambda_cycle

            # Generator identity loss
            id_loss_G = (
                self.identity_loss_fn(real_y, same_y)
                * self.lambda_cycle
                * self.lambda_identity
            )
            id_loss_F = (
                self.identity_loss_fn(real_x, same_x)
                * self.lambda_cycle
                * self.lambda_identity
            )

            # Total generator loss
            total_loss_G = gen_G_loss + cycle_loss_G + id_loss_G
            total_loss_F = gen_F_loss + cycle_loss_F + id_loss_F

            # Discriminator loss
            disc_X_loss = self.discriminator_loss_fn(disc_real_x, disc_fake_x)
            disc_Y_loss = self.discriminator_loss_fn(disc_real_y, disc_fake_y)

        # Get the gradients for the generators
        grads_G = tape.gradient(total_loss_G, self.gen_G.trainable_variables)
        grads_F = tape.gradient(total_loss_F, self.gen_F.trainable_variables)

        # Get the gradients for the discriminators
        disc_X_grads = tape.gradient(disc_X_loss, self.disc_X.trainable_variables)
        disc_Y_grads = tape.gradient(disc_Y_loss, self.disc_Y.trainable_variables)

        # Update the weights of the generators
        self.gen_G_optimizer.apply_gradients(
            zip(grads_G, self.gen_G.trainable_variables)
        )
        self.gen_F_optimizer.apply_gradients(
            zip(grads_F, self.gen_F.trainable_variables)
        )

        # Update the weights of the discriminators
        self.disc_X_optimizer.apply_gradients(
            zip(disc_X_grads, self.disc_X.trainable_variables)
        )
        self.disc_Y_optimizer.apply_gradients(
            zip(disc_Y_grads, self.disc_Y.trainable_variables)
        )

        return {
            "G_loss": total_loss_G,
            "F_loss": total_loss_F,
            "D_X_loss": disc_X_loss,
            "D_Y_loss": disc_Y_loss,
        }

**Callback:**
Periodically saves generated images.

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=2, save_interval=5):  # New parameter for interval
        self.num_img = num_img
        self.save_interval = save_interval

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_interval == 0:  # Only save every 20 epochs
            _, ax = plt.subplots(2, self.num_img, figsize=(12, 12))
            for i, img in enumerate(CelebA.shuffle(256).take(self.num_img)):
                prediction = self.model.gen_G(img)[0].numpy()
                prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
                img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

                ax[0, i].imshow(img)
                ax[1, i].imshow(prediction)
                ax[0, i].set_title("Input Image")
                ax[1, i].set_title("Translated Image")
                ax[0, i].axis("off")
                ax[1, i].axis("off")

                prediction = keras.utils.array_to_img(prediction)
                prediction.save(
                    "generated_img_{i}_{epoch}.png".format(i=i, epoch=epoch + 1)
                )
            plt.show()
            plt.close()

## Train the Model

In [ ]:
adv_loss_fn = keras.losses.MeanSquaredError()

In [ ]:
def generator_loss_fn(fake):
    fake_loss = adv_loss_fn(ops.ones_like(fake), fake)
    return fake_loss

In [ ]:
def discriminator_loss_fn(real, fake):
    real_loss = adv_loss_fn(ops.ones_like(real), real)
    fake_loss = adv_loss_fn(ops.zeros_like(fake), fake)
    return (real_loss + fake_loss) * 0.5

In [ ]:
model = GenshinfySiameseDataGenerator(
    generator_G=gen_G,
    generator_F=gen_F,
    discriminator_X=disc_X,
    discriminator_Y=disc_Y
)

In [ ]:
model.compile(
    gen_G_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    gen_F_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    disc_X_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    disc_Y_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    gen_loss_fn=generator_loss_fn,
    disc_loss_fn=discriminator_loss_fn,
)

In [ ]:
plotter = GANMonitor()
checkpoint_filepath = "./model_checkpoints/cyclegan_checkpoints.weights.h5"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, save_weights_only=True
)

In [ ]:
model.fit(
    CycleDataset.take(256),
    epochs=10,
    callbacks=[plotter, model_checkpoint_callback]
)